# Advent of code

### day 1

In [20]:
import pandas as pd

df = pd.read_csv("day1.csv", names=["values"])
print(df.shift(1).head())
ans = ((df - df.shift(1)) > 0).sum()[0]
print(f"The value increases {ans} times")

print("\n second \n")

print(df.rolling(3).sum().shift(-2).tail())

ans = ((df.rolling(3).sum().shift(-2) - df.rolling(3).sum().shift(-1)) > 0).sum()[0]
print(f"The value increases {ans} times")


   values
0     NaN
1   118.0
2   121.0
3   123.0
4   125.0
The value increases 1553 times

 second 

       values
1995  23858.0
1996  23872.0
1997  23881.0
1998      NaN
1999      NaN
The value increases 1597 times


# day 2

In [34]:
import pandas as pd
import numpy as np

dx = {
    "forward": 1,
    "back": -1,
    "down": 0,
    "up": 0,
}
dy = {
    "forward": 0,
    "back": 0,
    "down": -1,
    "up": 1,
}

moves = (
    pd.read_csv("day2.csv", sep=" ", names=["direction", "value"])
    .assign(dx=lambda df: df["direction"].replace(dx) * df["value"])
    .assign(dy=lambda df: df["direction"].replace(dy) * df["value"])
    [["dx", "dy"]]
)
print(moves.head(5))
print(f"The product of the final coordinates is {moves.sum().prod() * (-1)}")
moves = (
    moves
    .assign(aim=lambda df: df["dy"].cumsum() * (-1))
    .assign(dz=lambda df: (df["aim"] * df["dx"]))
    [["dx", "dz"]]
)
print(moves.head(5))
print(f"The product of the final coordinates is {moves.sum().prod()}")


   dx  dy
0   2   0
1   0  -7
2   0  -8
3   9   0
4   0  -8
The product of the final coordinates is 1670340
   dx   dz
0   2    0
1   0    0
2   0    0
3   9  135
4   0    0
The product of the final coordinates is 1954293920


# day 3

In [111]:
import pandas as pd
import numpy as np

bits = pd.read_csv("day3.csv", names=["value"], dtype=str)
p = (
    pd.DataFrame(bits["value"].apply(tuple).tolist())
    .astype(int)
    .mean()
    .round()
    .astype(int)
    .to_frame("gamma")
    .assign(epsilon=lambda df: 1 - df["gamma"])
    .astype(str)
    .sum()
    .apply(lambda x: int(x, 2))
    .prod()
)
print(f"The product of gamma and epsilon is {p}")

bits = pd.DataFrame(bits["value"].apply(tuple).tolist()).astype(int)
ix = bits.index
for col, values in bits.iteritems():
    v = values.loc[ix]
    common = int(round(v.mean() + 1e-6))
    ix = v[v == common].index
o2 = ix
ix = bits.index
for col, values in bits.iteritems():
    v = values.loc[ix]
    uncommon = int(1 - round(v.mean() + 1e-6))
    ix = v[v == uncommon].index if sum(v == uncommon) else ix
co2 = ix
q = (
    bits
    .loc[o2.union(co2), :]
    .astype(str)
    .sum(axis=1)
    .astype(int)
    .astype(str)
    .apply(lambda x: int(x, 2))
    .prod()
)
print(f"The product of o2 and co2 is {q}")

The product of gamma and epsilon is 749376
The product of o2 and c02 is 2372923


# day 4

In [81]:

import pandas as pd
import numpy as np
from io import StringIO

f = open("day4.txt", "r")
numbers = list(map(int, ''.join(f.readline()).strip().split(",")))
print("Sample:", numbers[:5])

boards = []
while True:
    board = ''.join(f.readline() for _ in range(6)).strip().replace("  ", " ").replace("\n ", "\n")
    if not board:
        break
    boards.append(pd.read_csv(StringIO(board), sep=" ", header=None))

print("Sample:\n", boards[3])

stop = False
for x in numbers:
    if stop:
        break
    for df in boards:
        df.replace(x, -1, inplace=True)
        if any((df == -1).sum(axis=0) == 5) or any((df == -1).sum(axis=1) == 5):
            print("Winning board:\n", df)
            print("x=", x)
            unmarked = df.sum().sum() + (df == -1).sum().sum()
            print("Sum of unmarked numbers:", unmarked)
            print("Product is:", x * unmarked)
            stop = True
            
stop_df = None
stop_x = None
for x in numbers:
    for i, df in enumerate(boards):
        if df is not None:
            df.replace(x, -1, inplace=True)
            if any((df == -1).sum(axis=0) == 5) or any((df == -1).sum(axis=1) == 5):
                stop_df = df
                stop_x = x
                boards[i] = None
print("Last winning board:\n", stop_df)
print("x=", stop_x)
unmarked = stop_df.sum().sum() + (stop_df == -1).sum().sum()
print("Sum of unmarked numbers:", unmarked)
print("Product is:", stop_x * unmarked)


Sample: [67, 31, 58, 8, 79]
Sample:
     0   1   2   3   4
0  87  49  12  85  88
1  67  89   7  35  70
2  37  45  93  84   9
3  80  58  54  13  22
4   8  71  48  15  39
Winning board:
     0   1   2   3   4
0  99  54  74  83  92
1  27  53  15  -1  -1
2  94  36  63  29  91
3  -1  -1  -1  -1  -1
4   9  95  23  98  33
x= 10
Sum of unmarked numbers: 1068
Product is: 10680
Last winning board:
     0   1   2   3   4
0  -1  49  71  59  90
1  -1  -1  -1  -1  -1
2  48  -1  -1  -1  -1
3  88  -1  -1  -1  -1
4  -1  -1  64  -1  -1
x= 68
Sum of unmarked numbers: 469
Product is: 31892


# day 5

In [54]:
import pandas as pd
import numpy as np
from io import StringIO
from functools import partial

f = open("day5.txt", "r")

ins = f.read().replace(" -> ", ",")
df = pd.read_csv(StringIO(ins), names=["x", "y", "r", "s"])
print("Sample:\n", df.head())
print("Number of vents: ", (df["x"] - df["r"].rename("x")).abs().sum() + (df["y"] - df["s"].rename("y")).abs().sum())
print("x:", min(df["x"].min(), df["r"].min()), max(df["x"].max(), df["r"].max()))
print("y:", min(df["y"].min(), df["s"].min()), max(df["y"].max(), df["s"].max()))

ocean = np.zeros((1000, 1000), dtype=int)

def line(arr, x, y, r, s, **kwargs):
    if x == r:
        if y < s:
            arr[x, y:(s+1)] += 1
        else:
            arr[x, s:(y+1)] += 1
    elif y == s:
        if x < r:
            arr[x:(r+1), y] += 1
        else:
            arr[r:(x+1), y] += 1        

for row in df.itertuples():
    line(arr=ocean, **row._asdict())
print("Number of locations with 2 or more vents:", (ocean >= 2).sum().sum())

ocean = np.zeros((1000, 1000), dtype=int)

def line(arr, x, y, r, s, **kwargs):
    if x == r:
        if y < s:
            arr[x, y:(s+1)] += 1
        else:
            arr[x, s:(y+1)] += 1
    elif y == s:
        if x < r:
            arr[x:(r+1), y] += 1
        else:
            arr[r:(x+1), y] += 1
    elif abs(x - r) == abs(y - s):
        for i, j in zip(range(x, r, 1 - 2 * (x > r)), range(y, s, 1 - 2 * (y > s))):
            arr[i][j] += 1
        arr[r][s] += 1

for row in df.itertuples():
    line(arr=ocean, **row._asdict())
print("Number of locations with 2 or more vents:", (ocean >= 2).sum().sum())


Sample:
      x    y    r    s
0  456  846  221  846
1  980  926   73   19
2  682  930  562  930
3  766  592  274  100
4  247  685  247   21
Number of vents:  252569
x: 10 989
y: 10 990
Number of locations with 2 or more vents: 5280
Number of locations with 2 or more vents: 16716


# day 6

In [17]:
import pandas as pd
import numpy as np
from numpy.linalg import matrix_power
from collections import Counter

f = open("day6.txt")
initial = list(map(int, f.read().strip().split(",")))
print("Sample:", initial[:5])
m = np.zeros((9, 9), dtype=int)

for i in range(8):
    m[i, i+1] = 1
m[6, 0] = 1
m[8, 0] = 1

fish = np.zeros((9,), dtype=int)

for i, j in Counter(initial).items():
    fish[i] = j
print("Number of fish after 80 days:", matrix_power(m, 80).dot(fish).sum())
print("Number of fish after 256 days:", matrix_power(m, 256).dot(fish).sum())

Sample: [3, 4, 1, 2, 1]
Number of fish after 80 days: 365131
Number of fish after 256 days: 1650309278600


# day 7

In [14]:
import pandas as pd
import numpy as np

f = open("day7.txt", "r")
crabs = list(map(int, f.read().strip().split(",")))
print("Sample:", crabs[:5])

pos = np.array(crabs, dtype=int)
a, b = pos.min(), pos.max() + 1
print("min", a, "max", b)
fuel = np.nan
ans = np.nan
for i in range(a, b):
    if not fuel < np.abs(pos - i).sum():
        ans = i
        fuel = np.abs(pos - i).sum()
print("Best horizontal:", ans)
print("Least fuel:", fuel)

fuel = np.nan
ans = np.nan
for i in range(a, b):
    if not fuel < (np.square(pos - i) + np.abs(pos - i)).sum() // 2:
        ans = i
        fuel = (np.square(pos - i) + np.abs(pos - i)).sum() // 2
print("Best horizontal:", ans)
print("Least fuel:", fuel)

Sample: [1101, 1, 29, 67, 1102]
min 0 max 1982
Best horizontal: 298
Least fuel: 356958
Best horizontal: 464
Least fuel: 105461913


# day 8

In [65]:
import pandas as pd
import numpy as np
from itertools import permutations
from functools import partial
from io import StringIO

with open("day8.txt", "r") as f:
    print("Number of 1, 4, 7, 8:", sum(len(x) in {2, 3, 4, 7} for line in f for x in line.split("|")[1].strip().split(" ")))

digits = {
    0: {0, 1, 2, 4, 5, 6},
    1: {2, 5},
    2: {0, 2, 3, 4, 6},
    3: {0, 2, 3, 5, 6},
    4: {1, 2, 3, 5},
    5: {0, 1, 3, 5, 6},
    6: {0, 1, 3, 4, 5, 6},
    7: {0, 2, 5},
    8: {0, 1, 2, 3, 4, 5, 6},
    9: {0, 1, 2, 3, 5, 6},
}
l_digits = list(zip(*sorted(digits.items())))[1]
print("Sample:\n", list(l_digits))
eligible = list(map(dict, map(partial(zip, tuple("abcdefg")), permutations(range(7)))))
print("Sample:\n", eligible[:5])
ans = 0
test = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""
# with StringIO(test) as f:
with open("day8.txt", "r") as f:
    for line in f:
        for candidate in eligible:
            stop = False
            for v in line.split("|")[0].strip().split(" "):
                if set(map(candidate.get, v)) not in l_digits:
                    stop = True
                    break
            if stop:
                continue
            else:
                ans += int(''.join(
                    map(
                        str,
                        map(
                            l_digits.index, 
                            map(
                                set, 
                                map(
                                    partial(map, candidate.get),
                                    line.split("|")[1].strip().split(" "),
                                )
                            )
                        )
                    )
                ))
                break
print("Sum:", ans)


Number of 1, 4, 7, 8: 521
Sample:
 [{0, 1, 2, 4, 5, 6}, {2, 5}, {0, 2, 3, 4, 6}, {0, 2, 3, 5, 6}, {1, 2, 3, 5}, {0, 1, 3, 5, 6}, {0, 1, 3, 4, 5, 6}, {0, 2, 5}, {0, 1, 2, 3, 4, 5, 6}, {0, 1, 2, 3, 5, 6}]
Sample:
 [{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 6, 'g': 5}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 5, 'f': 4, 'g': 6}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 5, 'f': 6, 'g': 4}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 6, 'f': 4, 'g': 5}]
Sum: 1016804


# day 9

In [114]:
import numpy as np
import pandas as pd
from collections import deque
from itertools import product

cave = pd.read_csv("day9.txt", header=None)[0].str.split("", expand=True).drop([0, 101], axis=1).astype(int).set_axis(range(100), axis=1)
print("Sample:\n", cave.head())
low = (
    (cave < cave.shift(-1, axis=0).fillna(np.inf)) &
    (cave < cave.shift(-1, axis=1).fillna(np.inf)) &
    (cave < cave.shift( 1, axis=0).fillna(np.inf)) &
    (cave < cave.shift( 1, axis=1).fillna(np.inf))
)
print("Numer of lows:", int((cave[low] + 1).sum().sum()))

visited = (cave == 9)
conn_idx = (cave == 9) * (-1)
conn = []
q = []
for i, p in enumerate(product(range(100), range(100))):
    if not visited.loc[p]:
        conn.append({p})
        conn_idx.loc[p] = i
        q.append((p, p))
    else:
        conn.append({})
while len(q) > 0:
    now, prev = q.pop()
    if not visited.loc[now]:
        visited.loc[now] = True
        for di, dj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
            if (0 <= now[0] + di < 100) and (0 <= now[1] + dj < 100) and conn_idx.loc[(now[0] + di, now[1] + dj)] != -1:
                q.append(((now[0] + di, now[1] + dj), now))
    a = conn_idx.loc[prev]
    b = conn_idx.loc[now]
    if b != a:
        big, small = (b, a) if len(conn[b]) > len(conn[a]) else (a, b)
        for p in conn[small]:
            conn_idx.loc[p] = big
        conn[big].update(conn[small])
        conn[small] == set()
l = list(sorted(map(len, conn), reverse=True))[:5]
print("Largest component sizes:", l[:5])
print("Product:", l[0] * l[1] * l[2])

Sample:
    0   1   2   3   4   5   6   7   8   9   ...  90  91  92  93  94  95  96  \
0   8   6   5   6   4   5   6   7   8   9  ...   7   7   9   7   6   5   5   
1   6   5   4   5   3   4   5   9   9   6  ...   5   6   7   9   5   4   3   
2   5   4   3   1   2   3   9   8   7   9  ...   6   7   8   9   6   5   4   
3   6   5   2   0   1   3   9   7   6   8  ...   7   8   9   9   8   7   5   
4   7   6   3   1   2   9   7   6   5   7  ...   8   9   6   9   9   8   7   

   97  98  99  
0   6   6   7  
1   1   5   6  
2   3   4   5  
3   4   5   6  
4   8   8   7  

[5 rows x 100 columns]
Numer of lows: 603
Largest component sizes: [94, 93, 90, 86, 86]
Product: 786780
